In [ ]:
import requests
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8888/v1/models/smart-grid-stability-model").json())

In [1]:
# import library
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [2]:
df = pd.read_csv('./data/smart_grid_stability.csv')

X = df.columns[:12].values
random = random.randint(0, len(X))
values = df.values[random]

inputs = {key: value for key, value in zip(X, values)}
inputs

{'tau1': 8.97170690932022,
 'tau2': 8.84842842134833,
 'tau3': 3.04647874898866,
 'tau4': 1.21451813833956,
 'p1': 3.40515818001095,
 'p2': -1.20745559234302,
 'p3': -1.27721014673295,
 'p4': -0.92049244093498,
 'g1': 0.163041039311334,
 'g2': 0.766688656526962,
 'g3': 0.839444015400588,
 'g4': 0.109853244952427}

In [ ]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, 'utf-8')]
        ),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )

In [ ]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result,
    })

In [ ]:
def make_predictions(inputs):
    response = requests.post(
        'https://stroke-detection.up.railway.app/v1/models/stroke-detection-model:predict',
        data=prepare_json(inputs)
    )

    predict = response.json()['predictions'][0][0]
    return predict

In [ ]:
predict = make_predictions(inputs)

print(f'{inputs}\n')
print(f'Unstable' if predict > 0.6 else f'Stable')